# Chain of Thought
This technique requires the language model to describe its solution in logical steps, and is particularly effective at improving answers to mathematical or reasoning questions. 

First, initialize the LLM object with your provider of choice.

In [ ]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key="KEY-ABC-DEF",
    temperature = 0,
    model_name='gpt-3.5-turbo'
)

This example shows a chain of thought prompt being used with few-shot, to illicit a similar output from the LLM.

In [ ]:
example = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.
"""

question = """
Q: The cafeteria had 23 apples.
If they used 20 to make lunch and bought 6 more, how many tennis balls do they have?
A:"""

llm.predict(example + '\n' + question)

This example further explores "Tree of thought", which combines Chain of Thought with a "Panel of Experts".

In [ ]:
# Based off https://github.com/dave1010/tree-of-thought-prompting

situation="""
Bob is in the living room.
He walks to the kitchen, carrying a cup.
He puts a ball in the cup and carries the cup to the bedroom.
He turns the cup upside down, then walks to the garden.
He puts the cup down in the garden, then walks to the garage.
Where is the ball?
"""

tree_of_thought="""
Imagine three different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.
The question is...
"""

print(llm.predict(tree_of_thought + '\n' + situation))

We can use Chain of Thought with vectors in our database as well:

In [ ]:
from rag_app.vector_search import VectorSearch

# Initiate the database connection
vector_database = VectorSearch(host='localhost', port=51972, namespace='USER', username='SuperUser', password='SYS')

# Load the cached text embedding model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("avsolatorio/GIST-Embedding-v0", cache_folder='.\\huggingface_cache')

In [ ]:
input = "How many manuscripts would the Vatican Library need to digitise over its 4 year digitisation project?"

embedding = model.encode(input)

similar_results = vector_database.search_by_story(embedding.tolist(), top_k=1)
document = similar_results

prompt = f"""
You are a useful assistant. 
These are examples of how you should answer your user's question by thinking through your steps very carefully: 
{example}

This is their question which you should answer, making sure you explain your steps carefully: 
{input}

These are supporting documents that you should use to form your answer to the user, which you can use in your steps: 
{document}
"""

print(prompt)

In [ ]:
llm.predict(prompt)